# Template Inspection Notebook

This notebook visualizes and analyzes the per-phase aortic velocity masks used as templates for deformable alignment.

## 1. Load and Display Templates
- Visual comparison
- Area and Center of Mass (COM)

## 2. Area and COM Statistics

## 3. Discussion
- Are templates smooth and circular?
- Are there large jumps in area or COM?

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass

template_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\templates\Template10allsimNonrigid"
template_prefix = "template_phase"
num_templates = 30

areas = []
centers = []

plt.figure(figsize=(20, 6))
for i in range(num_templates):
    path = os.path.join(template_dir, f"{template_prefix}_{i:02d}_similarity_10.npy")
    if not os.path.exists(path):
        print(f"Missing: {path}")
        continue
    mask = np.load(path)
    area = np.sum(mask)
    com = center_of_mass(mask)
    
    areas.append(area)
    centers.append(com)
    
    # Plot each template
    plt.subplot(3, 10, i + 1)
    plt.imshow(mask, cmap='gray')
    plt.title(f"{i:02d}\nA={int(area)}")
    plt.axis('off')

plt.suptitle("Template Masks (30 phases) - Area & Shape", fontsize=16)
plt.tight_layout()
plt.show()

# Plot area vs phase
plt.figure()
plt.plot(range(num_templates), areas, marker='o')
plt.title("Template Area per Phase")
plt.xlabel("Frame Index")
plt.ylabel("Area (pixels)")
plt.grid(True)
plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass

# مسیر تمپلیت‌ها
template_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\templates\Template10allsimNonrigid"

coms = []
areas = []
file_names = []

# بارگذاری و محاسبه COM و Area برای هر فاز
for i in range(30):
    tpl_path = os.path.join(template_dir, f"template_phase_{i:02d}_similarity_10.npy")
    if not os.path.exists(tpl_path):
        print(f"Missing template for phase {i}")
        continue
    tpl = np.load(tpl_path)
    tpl_bin = (tpl > 0.5).astype(np.uint8)
    
    com = center_of_mass(tpl_bin)
    area = np.sum(tpl_bin)
    
    coms.append(com)
    areas.append(area)
    file_names.append(f"Phase {i:02d}")

# جدا کردن مختصات مرکز جرم
com_y = [c[0] for c in coms]
com_x = [c[1] for c in coms]

# ---- رسم گراف‌ها ----
fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

axs[0].plot(file_names, areas, marker='o', label='Area')
axs[0].set_title("Template Area per Phase")
axs[0].set_ylabel("Area (pixels)")
axs[0].grid(True)

axs[1].plot(file_names, com_y, marker='s', color='green', label='COM Y')
axs[1].set_title("Center of Mass - Y")
axs[1].set_ylabel("Y Position")
axs[1].grid(True)

axs[2].plot(file_names, com_x, marker='d', color='orange', label='COM X')
axs[2].set_title("Center of Mass - X")
axs[2].set_ylabel("X Position")
axs[2].set_xlabel("Phase Index")
axs[2].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# template_inspection.ipynb
# Author: YourName
# Purpose: Visualize and analyze 30 per-phase aortic templates
# Notes: Run this to inspect area and COM (center of mass) across templates

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
import seaborn as sns

# ================================
# 1. Config
# ================================
template_dir = r"P:\Projects\DeepFlow\deepFlowDocker\scripts\Registration\templates\Template10allsimNonrigid"
threshold = 0.5

# ================================
# 2. Utility Functions
# ================================
def get_template_stats(mask):
    area = np.sum(mask)
    com = center_of_mass(mask)
    return {'area': int(area), 'com': com}
i
def show_template(mask, title='', threshold=0.5):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    ax[0].imshow(mask, cmap='gray')
    ax[0].set_title(f"Raw Template - {title}")

    bin_mask = (mask > threshold).astype(np.uint8)
    com = center_of_mass(bin_mask)
    ax[1].imshow(bin_mask, cmap='gray')
    ax[1].scatter(com[1], com[0], color='red', s=20, label='COM')
    ax[1].contour(bin_mask, levels=[0.5], colors='orange')
    ax[1].legend()
    ax[1].set_title(f"Binarized + COM - {title}")

    plt.suptitle(f"Template Visualization: {title}")
    plt.tight_layout()
    plt.show()

# ================================
# 3. Load and Analyze Templates
# ================================
templates = []
areas = []
coms = []
labels = []

for file in sorted(os.listdir(template_dir)):
    if file.endswith(".npy"):
        path = os.path.join(template_dir, file)
        arr = np.load(path)
        bin_mask = (arr > threshold).astype(np.uint8)

        stats = get_template_stats(bin_mask)
        areas.append(stats['area'])
        coms.append(stats['com'])
        labels.append(file)
        templates.append(bin_mask)

        show_template(arr, title=file)

# ================================
# 4. Plot Area and COM Stats
# ================================
areas = np.array(areas)
coms = np.array(coms)

fig, axs = plt.subplots(1, 3, figsize=(18, 5))
axs[0].plot(areas, marker='o')
axs[0].set_title("Template Area per Frame")
axs[0].set_xlabel("Frame")
axs[0].set_ylabel("Area (pixels)")

axs[1].plot(coms[:, 0], label='Y', marker='o')
axs[1].plot(coms[:, 1], label='X', marker='o')
axs[1].set_title("Center of Mass (COM) Coordinates")
axs[1].set_xlabel("Frame")
axs[1].set_ylabel("Pixels")
axs[1].legend()

axs[2].scatter(coms[:, 1], coms[:, 0], c=range(len(coms)), cmap='viridis')
axs[2].invert_yaxis()
axs[2].set_title("COM Trajectory in Image Space")
axs[2].set_xlabel("X")
axs[2].set_ylabel("Y")

plt.suptitle("Per-Frame Template Statistics")
plt.tight_layout()
plt.show()

# ================================
# 5. (Optional) Save COM & Area Data for Further Comparison with Test Sample
# ================================
import pandas as pd

template_stats_df = pd.DataFrame({
    'frame': list(range(len(areas))),
    'template_file': labels,
    'area': areas,
    'com_y': coms[:, 0],
    'com_x': coms[:, 1]
})

csv_out = os.path.join(template_dir, "template_stats.csv")
template_stats_df.to_csv(csv_out, index=False)
print(f"Template statistics saved to: {csv_out}")


In [ ]:
# Template Inspection for Per-Phase Aortic Velocity Masks
# This notebook/module compares all 30 template masks in terms of:
# 1. Area (number of foreground pixels)
# 2. Center of Mass (COM)
# 3. Visual representation: masks and their contours

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import center_of_mass
import seaborn as sns

# ------------------------ Configuration ------------------------
template_dir = r"P:\\Projects\\DeepFlow\\deepFlowDocker\\scripts\\Registration\\templates\\Template10allsimNonrigid"
template_prefix = "template_phase"
template_suffix = "similarity_10.npy"  # Format: template_phase_00_similarity_10.npy

# ------------------------ Load Templates ------------------------
templates = []
areas = []
coms = []

for i in range(30):
    fname = f"{template_prefix}_{i:02d}_{template_suffix}"
    path = os.path.join(template_dir, fname)
    if not os.path.exists(path):
        print(f"Warning: Missing template: {fname}")
        continue

    arr = np.load(path)
    binary = (arr > 0.5).astype(np.uint8)
    templates.append(binary)
    areas.append(np.sum(binary))
    coms.append(center_of_mass(binary))

# ------------------------ Plot All Templates ------------------------
def show_templates_side_by_side(templates, areas, coms):
    fig, axs = plt.subplots(5, 6, figsize=(18, 14))
    axs = axs.ravel()
    for i in range(len(templates)):
        axs[i].imshow(templates[i], cmap='gray')
        axs[i].scatter(coms[i][1], coms[i][0], c='red', s=20, label='COM')
        axs[i].set_title(f"Phase {i}\nArea: {areas[i]}\nCOM: ({coms[i][0]:.1f}, {coms[i][1]:.1f})")
        axs[i].axis('off')
    plt.suptitle("Template Masks with Area and Center of Mass", fontsize=18)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

show_templates_side_by_side(templates, areas, coms)

# ------------------------ Plot Area and COM Over Time ------------------------
def plot_area_and_com(areas, coms):
    coms_y = [c[0] for c in coms]
    coms_x = [c[1] for c in coms]

    fig, axs = plt.subplots(3, 1, figsize=(10, 10))

    axs[0].plot(areas, marker='o')
    axs[0].set_title("Template Mask Area (Foreground Pixels)")
    axs[0].set_xlabel("Phase")
    axs[0].set_ylabel("Area")

    axs[1].plot(coms_y, marker='o', label='Y')
    axs[1].set_title("Center of Mass - Y")
    axs[1].set_xlabel("Phase")
    axs[1].set_ylabel("Y")

    axs[2].plot(coms_x, marker='o', label='X', color='orange')
    axs[2].set_title("Center of Mass - X")
    axs[2].set_xlabel("Phase")
    axs[2].set_ylabel("X")

    plt.tight_layout()
    plt.show()

plot_area_and_com(areas, coms)

# ------------------------ Export Template Stats (Optional) ------------------------
template_stats = {
    f"phase_{i:02d}": {
        "area": areas[i],
        "com_y": float(coms[i][0]),
        "com_x": float(coms[i][1])
    } for i in range(len(templates))
}

# You can optionally save this as JSON or CSV for comparison with test masks later
#import json
#with open("template_stats.json", "w") as f:
#    json.dump(template_stats, f, indent=2)

print("Template inspection complete. You can now compare these stats with your test mask.")
